In [20]:
from azureml.core import Model
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.experiment import Experiment
from azureml.pipeline.wrapper import PipelineRun
from fasttext_pipeline_utils import choose_workspace, deploy_locally, deploy_to_ACI, deploy_to_AKS

In [2]:
# chose a workspace
subscription_id = '4f455bd0-f95a-4b7d-8d08-078611508e0b'
resource_group = 'fundamental'
workspace_name = 'fundamental3'
# set this if you have multiple tenant
tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47"
workspace=choose_workspace(subscription_id, resource_group, workspace_name, tenant_id)

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


name: fundamental3
resource_group fundamental
location eastasia
subscription_id 4f455bd0-f95a-4b7d-8d08-078611508e0b
compute_targets dict_keys(['myaks2', 'aml-compute', 'my-compute', 'compute-deploy'])


In [3]:
# get a PipelineRun object
experiment_name = "deploy"
experiment = Experiment(workspace, experiment_name)
run_id = '6acca15a-fcd4-446c-ab41-bd64494c67e3'
pipeline_run = PipelineRun(experiment, run_id)

In [7]:
# get a StepRun object
step_run = pipeline_run.find_step_run(name = 'FastText Train')[0]

In [12]:
# check ports
print('input ports:')
input_ports = step_run.input_ports
for port in input_ports:
    print(port,'\n')
output_ports = step_run.output_ports
print('\n\noutput ports:')
for port in output_ports:
    print(port,'\n')

input ports:
Port(Name:Training data dir,
Type:['AnyDirectory'],
StepRun:Run(Experiment: deploy,
Id: a07747b4-cfd9-4103-acb0-02f9fcaf4b83,
Type: azureml.StepRun,
Status: Completed)) 

Port(Name:Validation data dir,
Type:['AnyDirectory'],
StepRun:Run(Experiment: deploy,
Id: a07747b4-cfd9-4103-acb0-02f9fcaf4b83,
Type: azureml.StepRun,
Status: Completed)) 

Port(Name:Char2index dir,
Type:['AnyDirectory'],
StepRun:Run(Experiment: deploy,
Id: a07747b4-cfd9-4103-acb0-02f9fcaf4b83,
Type: azureml.StepRun,
Status: Completed)) 



output ports:
Port(Name:Trained model dir,
Type:['ModelDirectory'],
StepRun:Run(Experiment: deploy,
Id: a07747b4-cfd9-4103-acb0-02f9fcaf4b83,
Type: azureml.StepRun,
Status: Completed)) 



In [15]:
# download model from the output port of the Train module
port = step_run.get_port(name='Trained model dir')
saved_path = port.download(local_path='data', overwrite=True)
print('model save at: {}'.format(saved_path))

Downloaded azureml/a07747b4-cfd9-4103-acb0-02f9fcaf4b83/Trained_model_dir/BestModel, 1 files out of an estimated total of 1
model save at: data\azureml/a07747b4-cfd9-4103-acb0-02f9fcaf4b83/Trained_model_dir


In [18]:
# register model for deployment
model = Model.register(workspace, model_path=saved_path, model_name='model_for_deploy', tags={'deploy':1})

Registering model model_for_deploy


In [23]:
# register env if not registered before
name='env_for_deployment'
file_path='deployment/env_for_deployment.yaml'
env = Environment.from_conda_specification(name=name, file_path=file_path)
env = env.register(workspace=workspace)

In [24]:
# define inference configuration
entry_script='scoring_for_deployment.py'
version='1'
source_directory='deployment'
env = Environment.get(workspace=workspace, name=name, version=version)
inference_config = InferenceConfig(entry_script=entry_script, source_directory=source_directory,
                                       environment=env)

In [5]:
# deploy locally
service_name='local-deploy-test'
models=[model]
port=8892
service_locally = deploy_locally(workspace, service_name, models, inference_config, port=port)
print(service_locally.get_logs())

found the module of Split Data Txt
found the module of FastText Train
found the module of FastText Evaluation
found the module of FastText Score


In [ ]:
# deploy to ACI (Azure Container Instances)
# every time we deploy to ACI, we need to change the service_name or we delete the existing service beforehand
service_name='aci-deploy-test6'
models=[model]
service_aci=deploy_to_ACI(workspace, service_name, models, inference_config, cpu_cores=1, memory_gb=1, overwrite=True)

Running.........................................................................................................................................................

In [ ]:
# deploy to AKS (Azure Kubernetes Service)
# workspace with AKS
subscription_id = '74eccef0-4b8d-4f83-b5f9-fa100d155b22'
resource_group = 'DesignerDRI'
workspace_name = 'DesignerDRI_EASTUS'
# set this if you have multiple tenant
tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47"

workspace_aks=choose_workspace(subscription_id, resource_group, workspace_name, tenant_id)
attachment_name='myaks1'
# every time we deploy to ACI, we need to change the service_name or we delete the existing service beforehand
service_name='aks-deploy-test6'
models=[model]
token_auth_enabled=True
service_aks=deploy_to_AKS(workspace_aks, attachment_name, service_name, models, inference_config, token_auth_enabled=token_auth_enabled,
                  cpu_cores=1, memory_gb=1)